**Note to Self and the Community** - This is a request to anyone reading this kernel. Please **do not** spam the links of your notebooks in the comments or on any other notebook as it reduces the chances of other Kagglers to get recognised. I myself was guilty of this practise and i making ammendments now that i understand. So please provide your genuine feedbacks on the kernels and if you are commenting down i will definitely check out your notebooks. So keep Kaggling and help the community be more friendly and grow. Alright lets start with the notebook.

So in this Notebook we are taking a look at the Historic 911-calls data set and we will be carrying out our analysis and visualization on the data.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/911calls-historic-data/911.csv') #importing our data set

Now that we have our data set. Let's Take a look at our data.

In [ ]:
df.head() #checking the head of our data

Our data has total 9 columns and we can see we have data like Latitude, Longitude, etc Lets start by checking the top Zip codes and townsships in our data

In [ ]:
df['zip'].value_counts().head(5) #top 5 zip codes in our data

In [ ]:
df['twp'].value_counts().head(5) #top 5 townships in our data

In [ ]:
df['title'].nunique() #looking at the unique title codes

We can see in our data that we have a title column that describes what was the reason for the 911 call that was made. It has EMS,Fire and Traffic in it as the reasons. So lets make a new column to store these values.

In [ ]:
df['Reason'] = df['title'].apply(lambda title: title.split(':')[0]) #refining our titles column

In [ ]:
df['Reason'].value_counts() #checking for common reasons for 911 calls

So we can see that most common reason was  EMS when the call was made. Lets visualise this.

In [ ]:
sns.countplot(x='Reason',data=df,palette='inferno')

Now, lets check ur timestamp colun, we need our timestamp in date time format.

In [ ]:
type(df['timeStamp'].iloc[0])

Our timestamp section was of the type : str. We need to convert this to date time format and we will use to_datetime() method

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

Lets use the timestamp column to make new columns i.e Hour, Month and Day of week.

In [ ]:
df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
df['Month'] = df['timeStamp'].apply(lambda time: time.month)
df['Day of Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)

Lets map the days of the week to integers from 0-6.

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
df['Day of Week'] = df['Day of Week'].map(dmap)

Lets visualize the reasons for making the 911 calls on each day

In [ ]:
sns.countplot(x='Day of Week',data=df,hue='Reason',palette='inferno')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

Lets Do the same thing for months and lets see the results

In [ ]:
sns.countplot(x='Month',data=df,hue='Reason',palette='inferno')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

We are missing some months as we can see from the visulization. Lets do something about it

Lets create a groupby object called byMonth, where we group the DataFrame by the month column and use the count() method for aggregation.

In [ ]:
byMonth = df.groupby('Month').count()
byMonth.head()

Lets check the number of calls that were made per month

In [ ]:
byMonth['twp'].plot() #this plot helped us visualise the above months section by filling the mmissing data

Lets plot a linear fit to check number of calls during different months

In [ ]:
sns.lmplot(x='Month',y='twp',data=byMonth.reset_index()) #we need to reset our index for this plot

Lets create a new column called 'Date' that contains the date from the timeStamp column.

In [ ]:
df['Date']=df['timeStamp'].apply(lambda t: t.date())

Lets see the no. of calls using this Date column

In [ ]:
plt.figure(figsize = (8,6))
df.groupby('Date').count()['twp'].plot()
plt.tight_layout()

Next, we will visualize the call counts on basis of the different reasons i.e Traffic, EMS and Fire.

In [ ]:
plt.figure(figsize = (8,6))
df[df['Reason']=='Traffic'].groupby('Date').count()['twp'].plot()
plt.title('Traffic')
plt.tight_layout()

In [ ]:
plt.figure(figsize = (8,6))
df[df['Reason']=='Fire'].groupby('Date').count()['twp'].plot()
plt.title('Fire')
plt.tight_layout()

In [ ]:
plt.figure(figsize = (8,6))
df[df['Reason']=='EMS'].groupby('Date').count()['twp'].plot()
plt.title('EMS')
plt.tight_layout()

Lets Re-structure our data. We will make the hours as columns and the Days of the week will work as index

In [ ]:
dayHour = df.groupby(by=['Day of Week','Hour']).count()['Reason'].unstack()
dayHour.head()

Lets visualize this data using a heatmap and a clustermap

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayHour,cmap='inferno')

In [ ]:
sns.clustermap(dayHour,cmap='inferno')

Now lets make the months as our columns

In [ ]:
dayMonth = df.groupby(by=['Day of Week','Month']).count()['Reason'].unstack()
dayMonth.head()

Lets make Heatmap and Clustermap to visualize this data

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayMonth,cmap='inferno')

In [ ]:
sns.clustermap(dayMonth,cmap='inferno')

To be continued...